# User Experience Analytics

## Imports

In [1]:
import pickle
import numpy as np
import pandas as pd
from math import floor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import zscore
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_selector import DfSelector
from df_outlier import DfOutlier
from vis_seaborn import *
from vis_plotly import *

2021-07-17 02:51:14,225 — DfSelector — DEBUG — Loaded successfully!
2021-07-17 02:51:14,230 — DfOutlier — DEBUG — Loaded successfully!


## Data reading

In [4]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146882 entries, 0 to 146881
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   bearer_id                       146882 non-null  int64  
 1   start                           146882 non-null  object 
 2   start_ms                        146882 non-null  float64
 3   end                             146882 non-null  object 
 4   end_ms                          146882 non-null  float64
 5   dur_(ms)                        146882 non-null  float64
 6   imsi                            146882 non-null  int64  
 7   msisdn_number                   146882 non-null  int64  
 8   imei                            146882 non-null  int64  
 9   last_location_name              146882 non-null  object 
 10  avg_rtt_dl_(ms)                 146882 non-null  float64
 11  avg_rtt_ul_(ms)                 146882 non-null  float64
 12  avg_bearer_tp_dl

## Task 4.1

engagement score to each user. Consider the engagement score as the Euclidean distance between the user data point & the less engaged cluster(use the first clustering for this)(Euclidean Distance)


experience score to each user. Consider the experience score as the Euclidean distance between the user data point & the worst experience’s cluster.


## Task 4.2

Consider the average of both engagement & experience scores as the satisfaction score & report the top 10 satisfied customer


## Task 4.3


Build a regression model of your choice to predict the satisfaction score of a customer.


## Task 4.4


Run a k-means(k=2) on the engagement & the experience score .


## Task 4.5


Aggregate the average satisfaction & experience score per cluster.


## Task 4.6


Export your final table containing all user id + engagement, experience & satisfaction scores in your local MySQL database. Report a screenshot of a select query output on the exported table.


## Task 4.7


Model deployment tracking - deploy the model and monitor your model. Here you can use MlOps tools which can help you to track your model’s change.  Your model tracking report includes code version, start and end time, source, parameters, metrics(loss convergence) and artifacts or any output file regarding each specific run. (CSV file, screenshot)
